In [7]:
import heapq #El módulo heapq implementa colas de prioridad (heaps)

In [8]:
class Node:
    def __init__(self, position, parent=None, action=None,path_cost=0): #AGREGAR ACTION
        self.position = position
        self.parent = parent
        self.path_cost = path_cost
        self.action=action

    def __lt__(self, other):
        return self.path_cost < other.path_cost

class Problem:
    def __init__(self, maze, start, end,actions):
        self.maze = maze
        self.start = start
        self.end = end
        self.actions=actions
        

In [9]:
def reconstruct_path(node):  #AJUSTAR FUNCIONES PARA ADEMAS DE LAS POSICIONES, MOSTRAR LAS ACCIONES TOMADAS
    path = []
    actions=[]
    while node:
        path.append(node.position)
        if node.action:
            actions.append(node.action)
        node = node.parent
    path.reverse()
    actions.reverse()
    return path, actions


In [10]:
def find_exit(maze):
    start = (1, 1)  # Posición inicial basado en la documentación suministrada
    end = (1, 6)    # Posición de la salida basado en la documentación suministrada

    #DEFINA el conjunto de actions posibles#
    actions = {
            "up": (-1, 0),
            "down": (1, 0),
            "left": (0, -1),
            "right": (0, 1)
        }

    problem=Problem(maze,start,end,actions)#COMPLETE LA DEFINICIÓN DEL OBJETO Y ADAPTELO EN LOS PUNTOS QUE LO REQUIERAN

    def manhatan_distance(pos, goal):
        return abs(pos[0] - goal[0]) + abs(pos[1] - goal[1])  # Distancia de Manhattan

    def get_neighbors(pos):  #ESTA ES LA FUNCIÓN QUE DEBERIA AJUSTAR PARA HACER TRACKING DE LOS MOVIMIENTOS (Up, Down, Right, Left)
        neighbors = [] #lista de vecinos
        for action, (dx, dy) in problem.actions.items():  #Tenga en cuenta que para que esto sea funcional ya debio haber definido el objeto problem
            neighbor = (pos[0] + dx, pos[1] + dy)
            if maze[neighbor[0]][neighbor[1]] != "#": #si el vecino es diferente a "#" pared agregarlo a la lista de vecinos                neighbors.append(neighbor)
              neighbors.append((neighbor,action))
        return neighbors

    start_node = Node(start, path_cost=0)
    frontier = [(manhatan_distance(start, end), start_node)]
    heapq.heapify(frontier) #Convierte la lista frontier en una cola de prioridad (heap)
    reached = {start: start_node}

    while frontier:
        _, node = heapq.heappop(frontier)
        if node.position == end:
            return reconstruct_path(node)

        for neighbor, action in get_neighbors(node.position):
            new_cost = node.path_cost + 1
            if neighbor not in reached or new_cost < reached[neighbor].path_cost:
                reached[neighbor] = Node(neighbor, parent=node,action=action, path_cost=new_cost)
                heapq.heappush(frontier, (manhatan_distance(neighbor, end), reached[neighbor]))

    return None  # No se encontró salida

In [11]:
maze = [
    ["#", "#", "#", "#", "#", "#", "#","#"],
    ["#", "S", "#", " ", "#", " ", "E","#"],
    ["#", " ", " ", " ", "#", " ", " ","#"],
    ["#", " ", "#", " ", " ", " ", "#","#"],
    ["#", "#", "#", "#", "#", "#", "#","#"],
    ["#", "#", "#", "#", "#", "#", "#","#"]
]
path = find_exit(maze)
print("Path to exit:", path)

Path to exit: ([(1, 1), (2, 1), (2, 2), (2, 3), (3, 3), (3, 4), (3, 5), (2, 5), (1, 5), (1, 6)], ['down', 'right', 'right', 'down', 'right', 'right', 'up', 'up', 'right'])


**UNA VEZ SOLUCIONADO EL EJERCICIO, RESPONDA:**

#### **1. ¿Cómo cambia el comportamiento del algoritmo si cambiamos la función de costo?**
##### Actualmente la función de costo se da gracias a la heurística que nos proporciona la distancia de Manhatan, con esta podemos elegir el nodo (en este caso, la casilla del laberinto) con la “distancia más corta” hasta el final. Si bien la distancia Manhatan siempre toma el mejor de los casos, es decir, un camino sin obstáculos, es gracias a esta que podemos priorizar los nodos más cercanos hacía el objetivo. Más allá de esto, el costo de un nodo vecino se calcula incrementando en uno el costo del nodo del cual se es vecino y preguntándose si este es el menor de los costos que se tiene hasta ahora. 

##### Ahora bien, con esto claro, la función de costos puede verse principalmente afectada si cambiamos la manera en la que los nodos vecinos son priorizados para ser visitados. Así, podría pasar que ya no usáramos un heapq para tomar primero el nodo con la distancia relativa más corta al final, en este caso, el algoritmo ya no elegiría primero el nodo que piensa es el más cercano, si no que iniciaría por cualquiera de los nodos vecinos y lo exploraría, lo cual también nos proporcionaría un camino hacia la salida, pero no necesariamente el más cercano, lo que podría hacer al algoritmo ineficiente, pues estaría tomando más pasos para hallar la ruta, es decir, terminaría por ser un algoritmo de fuerza bruta. 

##### Por otro lado, también podría hacerse un cálculo distinto a la distancia de Manhatan para realizar la priorización de nodos, como, por ejemplo, la distancia euclidiana, lo cual  permitiría un flujo del algoritmo prácticamente igual, solo que con un parámetro distinto para elegir los nodos a ser primeramente visitados, sin embargo, esto podría terminar por hacer que el algoritmo no sea tan efectivo, pues esta distancia no es tan apropiada cuando no es posible desplazarse en diagonal, brindándonos un resultado que no necesariamente es el camino más corto.

#### **2. ¿Qué sucede si hay múltiples salidas en el laberinto? ¿Cómo podrías modificar el algoritmo para manejar esta situación?**
##### Primeramente, en caso de contar con más de una salida, ya no tendríamos un solo objetivo si no, posiblemente más de uno, esto impide usar la distancia de Manhatan como lo veníamos haciendo, pues deberían considerarse las distancias relativas de los nodos a todos los posibles finales.

#### Para solucionar esto, hemos planteado dos principales opciones: 
#### -	La primera descarta a idea de priorizar los nodos, es decir, este simplemente intentaría llegar a alguna de las salidas, y si no la encuentra con el primer camino probado haría Backtrack, lo que terminaría por ser un algoritmo de fuerza bruta.
 
#### -	La segunda sería primeramente ingresar todos los posibles finales en una lista, elegir el primero como el único final, luego se procedería como en lo habitual, encontrando la ruta más corta del único inicio al “único final”, después se actualizaría el costo del camino de este final y se sacaría de la cola de prioridad, este mismo proceso se haría con cada uno de los finales dentro de la cola de prioridad, y finalmente se elegiría el final con el menor costo. Otra opción sería priorizar los finales por su distancia de Manhathan con el inicio, y parar al encontrar la ruta más corta con el primer final (si este no tiene ruta posible, se procedería con el segundo final priorizado y así sucesivamente).

### Cabe aclarar que para las ultimás dos opciones, es necesario que los otros finales, ajenos al que estoy analizando, se puedan considerar como posibles partes del camino. 

#### **3. Modifica el laberinto por uno más grande y con otros tipos de obstáculos, además de paredes. ¿Qué limitaciones encuentras en el algoritmo?**

### El principal problema es que el algoritmo no identifica más obstaculos, solo lás paredes, por lo que tomaría los obstáculos como posibles vecinos (celdas a las que se puede avanzar) terminando por darnos rutas no válidas. Con respecto al tamaño su mayor limitación es que entre más grande sea el laberinto más nodos tendrá que explorar y al tener que guardar en la cola todos los nodos visitados podría incurrir en un uso excesivo en la memoria y un tiempo más largo de procesamiento. Además, tanto el inicio como el final deberían de calcularse y no entrar sus cordenadas manualmente, pues en caso de tener más finales, este proceso resulta tardío.


Nota: Resuelve este problema en una celda aparte para mantener la integridad de tu código original.

In [12]:
maze2 = [
    ["/", "#", "#", "#", "#", "/", "#", "*", "#", "#", "#", "/"],
    ["#", "S", "#", " ", "#", " ", " ", " ", " ", "*", "#", "#"],
    ["#", " ", " ", " ", "#", " ", "#", " ", "/", "#", "#", "#"],
    ["#", " ", "*", " ", " ", " ", "*", " ", "#", "#", "#", "#"],
    ["*", "#", "#", "#", "*", "#", "#", " ", " ", "E", "#", "#"],
    ["#", "#", "#", "#", "#", "/", "#", " ", "#", "#", "#", "#"],
    ["#", "#", "*", "/", "#", "#", "#", " ", "#", "#", "*", "#"],
    ["*", " ", " ", " ", "#", "*", "*", " ", "#", "#", "#", "#"],
    ["#", " ", "#", " ", " ", " ", " ", " ", "#", "#", "#", "#"],
    ["/", " ", "#", "#", "#", "#", "#", "#", "#", "#", "#", "/"]
]
path = find_exit(maze2)
print("Path to exit:", path)

Path to exit: ([(1, 1), (2, 1), (2, 2), (2, 3), (3, 3), (3, 4), (3, 5), (2, 5), (1, 5), (1, 6)], ['down', 'right', 'right', 'down', 'right', 'right', 'up', 'up', 'right'])
